In [29]:
import arff

from scipy import sparse
decoder = arff.ArffDecoder()
d = decoder.decode(open('rcv1subset1-train.arff', 'r'), encode_nominal=True, return_type=arff.COO)
data = d['data'][0]
row = d['data'][1]
col = d['data'][2]
matrix = sparse.coo_matrix((data, (row, col)), shape=(max(row)+1, 47337))
d = decoder.decode(open('rcv1subset1-test.arff', 'r'), encode_nominal=True, return_type=arff.COO)
data = d['data'][0]
row = d['data'][1]
col = d['data'][2]
test_matrix = sparse.coo_matrix((data, (row, col)), shape=(max(row)+1, 47337))



In [30]:
import numpy as np
import torch
import argparse
result = {}

# set up instance-feature and instance-label matrix
src=matrix.A[:,0:47236]
tgt = matrix.A[:,47236:]
src[np.nonzero(src)] = 1 # turn feature vector to binary
src = src.astype(np.int32)
tgt = tgt.astype(np.int32)

In [31]:
# ADDED MECHANISM TO CHOOSE TOP FEATURES
print(np.sort(src.sum(axis=0))[::-1])
print(np.argsort(src.sum(axis=0))[::-1])
top_indices = np.argsort(src.sum(axis=0))[::-1][0:4000]
src = src[:, top_indices]

[1332 1063 1010 ...    0    0    0]
[46544 31593 28752 ... 28363 28359     0]


In [32]:

train = {}

src_as_seq = []
for i in range(src.shape[0]): # MrMP takes a sequence as input, not size-num_label binary array. So a conversion is performed here. 
    src_as_seq.append((np.nonzero(src[i])[0]).tolist())
tgt_as_seq = [] # list containing one list of labels for each instance
for i in range(tgt.shape[0]):
    # find the indices where the label occurs, add 4 to the indices 
    # (since the MrMP model reserves 0,1,2,3 for special labels), 
    # and append the label list to the result
    tgt_as_seq.append((np.nonzero(tgt[i])[0]+4).tolist()) # We add 4 to the label index, since the MrMP model reserves 0,1,2,3 for special labels

# select random instances for validation
valid_index = np.random.choice(len(src_as_seq), int((len(src_as_seq)+len(tgt_as_seq))*0.1), replace=False) 
valid_from_src = np.array(src_as_seq,dtype=object)[valid_index].tolist()
valid_from_tgt = np.array(tgt_as_seq,dtype=object)[valid_index].tolist()
print(len(src_as_seq))
print(len(valid_index))
src_as_seq = np.delete(src_as_seq,valid_index,0).tolist()
tgt_as_seq = np.delete(tgt_as_seq,valid_index,0).tolist()
print(len(src_as_seq))
train['src']=src_as_seq
train['tgt'] = tgt_as_seq
result['train'] = train

result['valid'] = {}
result['valid']['src'] = valid_from_src
result['valid']['tgt'] = valid_from_tgt


src=test_matrix.A[:,0:47236]
tgt = test_matrix.A[:,47235:-1]
src[np.nonzero(src)] = 1
src = src.astype(np.int32)
tgt = tgt.astype(np.int32)

# CHOSE TOP FEATURES
src = src = src[:, top_indices]

test = {}

src_as_seq = []
for i in range(src.shape[0]):
    src_as_seq.append(np.nonzero(src[i])[0].tolist())
tgt_as_seq = []
for i in range(tgt.shape[0]):
    tgt_as_seq.append((np.nonzero(tgt[i])[0]+4).tolist())
#print(src_as_seq)


test['src']=src_as_seq
test['tgt'] = tgt_as_seq
result['test'] = test

# in the .arff format, the original dictionary is not provided. We will create a placeholder dictionary

src_dictionary = {str(i): i for i in range(src.shape[1])}

tgt_dictionary = {str(i): (i+4) for i in range(tgt.shape[1])}
tgt_dictionary['<blank>'] = 0
tgt_dictionary['<unk>'] = 1
tgt_dictionary['<s>'] = 2
tgt_dictionary['</s>'] = 3
print(tgt.shape[1])
result['dict'] = {}
result['dict']['src'] = src_dictionary
result['dict']['tgt'] = tgt_dictionary


3000
600
2400
101


In [33]:
result['settings'] = argparse.Namespace()


torch.save(result,'train_valid_test_4000.pt')

In [15]:
src.shape

(3000, 2000)

In [17]:
maximum = 0
for list_item in result['train']['tgt']:
    for num in list_item:
        maximum = max(maximum,num)
print(maximum)
print(result['train']['tgt'])

104
[[5, 37, 74, 75], [5, 6, 37, 74, 75, 82], [8, 9, 11, 37], [8, 9, 10, 37], [8, 11, 18, 19, 37], [12, 13, 37], [6, 13, 25, 37, 74, 78, 95], [13, 14, 37], [13, 15, 37], [6, 13, 16, 25, 37], [13, 17, 37], [18, 20, 37], [18, 19, 37], [13, 15, 18, 20, 37], [18, 21, 37], [22, 26, 37, 102], [22, 25, 37], [26, 37], [6, 26, 27, 28, 37, 57, 59, 63, 102, 103], [4, 26, 28, 29, 37], [8, 11, 26, 29, 37], [30, 37], [6, 31, 37], [31, 32, 37, 74, 76], [6, 33, 37], [34, 35, 37], [18, 19, 36, 37, 55, 63, 74, 83], [8, 9, 37], [38, 39, 48, 49, 63, 64, 68, 74], [38, 39, 57, 63], [39, 40, 63, 99, 100], [41, 43, 63], [41, 43, 63], [44, 45, 55, 63, 74, 83], [44, 45, 55, 63, 74, 83], [44, 46, 63], [38, 39, 44, 47, 63], [48, 49, 63], [39, 48, 49, 63, 64, 68, 74], [48, 50, 63], [51, 52, 63], [22, 25, 37, 51, 52, 53, 63], [51, 54, 63], [6, 36, 37, 55, 63, 74, 83], [55, 56, 63], [57, 59, 63, 74, 77], [57, 58, 63, 74, 92], [57, 59, 63], [57, 60, 63], [26, 37, 61, 63], [62, 63], [55, 56, 63], [64, 65, 74, 87], [6,

In [2]:
import numpy as np
tgt_binary = list()
tgt_seq = [[3,10],[2,4,10]]
num_labels = 11
for i in range(len(tgt_seq)): # go through every article
    y = np.zeros(num_labels) 
    y[tgt_seq[i]] = 1
    tgt_binary.append(y)
print(tgt_binary)

[array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]), array([0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1.])]


In [4]:
temp = [3,9,0,4,6]
np.delete()

1

In [9]:
[1,3]+3

TypeError: can only concatenate list (not "int") to list